In [1]:
using LinearAlgebra
using Test

In [2]:
# Defining parameters:
α = 10e-4;
β = 10e-3;
γ = 10e-6;
σ = 0.5;
ρ = 10e-3;
MAX = 1000;
ϵ = 10e-6;

In [31]:
function compare( A::Matrix{Float64}, B::Matrix{Float64} )
    # True: if A equals B
    # False: otherwise.
    ϵ = 10e-6;
    size_A = size(A);
    size_B = size(B);
    if( length(size_B) != length(size_A) )
        return false
    end
    for i in size_A[1]
        for j in size_A[2]
            if( !(B[i,j] - ϵ ≤ A[i, j] ≤ B[i, j] + ϵ) )
                return false
            end
        end
    end
    
    return true
end

compare (generic function with 1 method)

In [36]:
A = zeros(2,2)
B = zeros(2,2)
println(@test compare(A,B))
A = ones(2,2)
B = ones(2,2)
println(@test compare(A, B))

Test Passed
Test Passed


In [3]:
# Defining the quadratic formula:
function quadratic( vector::Vector{Float64}, flag::Bool )
    isempty(vector) && throw(ArgumentError("Input is empty!"));
    n = length(vector);
    if( flag )
        # It will store gradient info.http://localhost:8888/notebooks/projeto-computacional.ipynb
        grad = zeros(Float64, (n, 1));
    end
    # It will store function value.
    sum = 0.0;
    
    for i in eachindex(vector)
        sum += i * vector[i] * vector[i];
        if( flag )
            grad[i] = 2 * i * vector[i];
        end
    end

    if( flag )
        return sum, grad;
    else
        return sum;
    end
end

quadratic (generic function with 1 method)

In [4]:
function quadratic_hessian( x::Vector{Float64} )
    N = length(x);
    Hessian = Diagonal( 2.0 * (1:N) )
    return Hessian;
end

quadratic_hessian (generic function with 1 method)

In [5]:
println(@test quadratic_hessian([1.0; 2.0]) == [2.0 0.0; 0.0 4.0] )
println(@test quadratic_hessian([1.0; 3.0]) == [2.0 0.0; 0.0 4.0] )
println(@test quadratic_hessian([0.0; 2.0]) == [2.0 0.0; 0.0 4.0] )
println(@test quadratic_hessian([1.0; 3.0; 4.0]) == [2.0 0.0 0.0; 0.0 4.0 0.0; 0.0 0.0 6.0])

Test Passed
Test Passed
Test Passed
Test Passed


In [6]:
# Testing the quadratic formula:
println(@test quadratic( [1.0; 2.0; 3.0], false ) == 36.0);
println(@test quadratic( [1.0; 2.0; 3.0; 4.0], false ) == 100.0);
println(@test quadratic( [1.0; 2.0; 3.0; 4.0; 5.0], false ) == 225.0);
println(@test quadratic( [0.0; 0.0; 0.0; 0.0; 0.0], false ) == 0.0);
println(@test  (2.5502 - ϵ <= quadratic( [0.99; 0.73; 0.41], false ) <= 2.5502 + ϵ ) );

Test Passed
Test Passed
Test Passed
Test Passed
Test Passed


In [7]:
# Defining the rosenbrok function and it's gradient.
function rosenbrok( vector::Vector{Float64}, flag::Bool )
    n = length(vector);
    ( n % 2 == 1 ) && throw(ArgumentError("Input is not even."));
    if( flag )
        grad = zeros(Float64, (n, 1));        
    end
    
    sum = 0.0;
    n = n / 2;
    n = convert(Int64, n);
    for i = 1:n
        xᵢ = vector[2*i - 1];
        xₚ = vector[2*i];
        aux1 = xₚ - xᵢ * xᵢ;
        aux2 = xᵢ - 1;

        if( flag )
            grad[2*i - 1] = -40.0 * aux1 * xᵢ;
            grad[2*i] = 20.0 * aux1;
        end

        sum = sum + 10 * aux1 * aux1 + aux2 * aux2;        
    end
    
    if( flag )
        return sum, grad;
    else
        return sum;
    end
end

rosenbrok (generic function with 1 method)

In [8]:
function rosenbrok_hessian( x::Vector{Float64} )
    N = length(x);
    Hessian = zeros(Float64, N, N);
    iter = convert(Int64, N / 2 );
    for i = 1:iter
        even = 2 ⋅ i;
        odd = even - 1;
        Hessian[odd, odd] = -40*x[even] + 120 ⋅ x[odd] ⋅ x[odd] + 2;
        Hessian[even, even] = 20*x[even];
        Hessian[even, odd] = Hessian[odd, even] = -40 ⋅ x[odd];
    end
    
    return Hessian;
end

rosenbrok_hessian (generic function with 1 method)

In [9]:
matrix = [42.0  -40.0  0.0  0.0;
          -40.0  40.0  0.0  0.0;
           0.0   0.0  922.0  -120.0;
           0.0   0.0  -120.0  80.0]

println(@test rosenbrok_hessian([1.0; 2.0; 3.0; 4.0]) == matrix)


Test Passed


In [10]:
# testing the Rosenbrok function:
println( @test rosenbrok([1.0,1.0,1.0,1.0], false ) == 0.0 )


Test Passed


In [11]:
# Defining the Styblinsky-Tang function
function styblinsky( vector::Vector{Float64}, flag::Bool )
    isempty(vector) && throw(ArgumentError("Vector is empty."));
    n = length(vector);
    if( flag )
        grad = zeros(Float64, (n, 1));
    end
    sum = 0.0;
    
    for i in eachindex(vector)
        xᵢ = vector[i];
        sum = sum + xᵢ ^ 4 - 16 * xᵢ ^ 2 + 5 * xᵢ;

        if( flag )
            grad[i] = 4 * xᵢ ^ 3 - 32 * xᵢ + 5;
        end
    end

    if( flag )
        return sum, grad;
    else
        return sum;
    end
end

styblinsky (generic function with 1 method)

In [12]:
function styblisnky_hessian( x::Vector{Float64} )
    N = length(x);
    hessian = zeros(Float64, N, N);
    for i in eachindex(x)
        hessian[i, i] = 12 ⋅ x[i] ⋅ x[i] - 32;
    end
    
    return hessian;
end

styblisnky_hessian (generic function with 1 method)

In [13]:
println(@test styblisnky_hessian([1.0;2.0;3.0]) == [-20.0 0.0 0.0; 0.0 16.0 0.0; 0.0 0.0 76.0])

Test Passed


In [14]:
# testing the Styblinsky-Tang function:
println(@test -78.322 * 2.0 - 10e-2 <= styblinsky( [-2.903534; -2.903534], false ) <= -78.322 * 2.0 + 10e-2 )

Test Passed


In [12]:
# Defining the Rastrigin function:
function rastringin( vector::Vector{Float64}, flag::Bool )
    isempty(vector) && throw(ArgumentError("Vector is empty."));
    n = length(vector);
    if( flag )
        grad = zeros(Float64, (n, 1));
    end
    sum = 0.0;

    for i in eachindex(vector)
        xᵢ = vector[i];
        aux = 2 * π;
        if( flag )
            grad[i] = 2 * xᵢ + 10 * sin( aux * xᵢ ) * aux;
        end

        sum = sum + xᵢ * xᵢ - 10 * cos( aux * xᵢ );
    end


    if( flag )
        return sum, grad;
    else
        return sum;
    end
end

rastringin (generic function with 1 method)

In [15]:
function rastring_hessian( x::Vector{Float64} )
    N = length(x);
    hessian = zeros(Float64, N, N);
    for i = 1:N
        hessian[i,i] = 40 ⋅ π ^ 2 ⋅ cos( 2⋅π⋅x[i] ) + 2;
    end
    
    return hessian;
end

rastring_hessian (generic function with 1 method)

In [25]:
aux = 1/(2π)⋅acos(-2/(40⋅π⋅π))
rastring_hessian([aux; aux; aux])

3×3 Matrix{Float64}:
 6.43929e-15  0.0          0.0
 0.0          6.43929e-15  0.0
 0.0          0.0          6.43929e-15

In [38]:
# Testing the rastring_hessian function
aux = 1/(2π)⋅acos(-2/(40⋅π⋅π));
A = zeros(2,2);
B = rastring_hessian([aux; aux]);
println(@test compare(A,B))
A = zeros(3, 3);
B = rastring_hessian([aux; aux; aux]);
println(@test compare(A,B))

Test Passed
Test Passed


In [13]:
# Testing the rastringin function:
println( @test rastringin([0.0; 0.0], false) == -20 )
println( @test rastringin([0.0; 0.0; 0.0], false) == -30 )

Test Passed
Test Passed


In [48]:
function gradient_method( α::Float64, σ::Float64, ϵ::Float64, M::Int64, x₀::Vector{Float64}, 
         f::Function, display::Bool )
    k = 0;
    xₖ = x₀;
    evaluated_function, evaluated_grad = f(x₀, true);  
    # ERROR: opnorm is not the infty norm (corrected)
    while( norm(evaluated_grad, Inf) >= ϵ && k < M )
        descent_direction = -1 ⋅ evaluated_grad;
        tₖ = 1;
        # Don't know if norm is actually working.... ERROR. (corrected)
        armijo = - α * norm( evaluated_grad, 2 )
        while( f( xₖ + tₖ * descent_direction, false ) > evaluated_function + tₖ * armijo )
            tₖ = tₖ * σ;
        end
        
        xₖ = xₖ + tₖ * descent_direction;
        evaluated_function, evaluated_grad = f(xₖ, true);
        k  = k + 1;
        if( display )
            println("Iter: $k");
            println("Value of function: $evaluated_function");
            println("Curret point:");
            display(xₖ);
            println("Curret Gradient: ");
            display(evaluated_grad);
        end
    end
    
    return xₖ;
end

gradient_method (generic function with 1 method)

In [63]:
function newton_method( α::Float64, 
        β::Float64, 
        γ::Float64, 
        σ::Float64, 
        ρ::Float64, 
        ϵ::Float64, 
        M::Int64, 
        x::Vector{Float64}, 
        f::Function, 
        hessian_function::Function, 
        display::Bool
    )
    N = length(x);
    eval_x, eval_grad = f(x, true);
    xₖ = x;
    k = 0;
    while( norm(eval_grad, Inf) ≥ ϵ && k < M )
        μ = 0;
        flag_aux = true;
        # Try to solve the descent direction system for newton's method:
        hessian = hessian_function( xₖ );
        norm_grad = norm( eval_grad, 2);
        while( flag_aux )
            matrix = hessian + μ ⋅ I(N);
            try
                dₖ = linalg.solve( matrix, -1 ⋅ eval_grad );
                aux = dot( eval_grad, dₖ ) / ( norm_grad ⋅ norm( dₖ, 2 ) );
                if( aux ≤ -γ )
                    flag_aux = true;
                else
                    μ = max( 2⋅μ, ρ );
                end
            catch ex
                μ = max( 2⋅μ, ρ );
            end
        end
        
        norm_desc = norm(dₖ,2);
        if( norm_desc < β⋅norm_grad )
             constante = β ⋅ norm_grad / norm_desc;
             dₖ = constante ⋅ dₖ;
        end
        
        tₖ = 1;
        function_value = f(xₖ, false);
        armijo_condition = α ⋅ dot( eval_grad, dₖ );
        while( f(xₖ + tₖ⋅dₖ) > function_value + tₖ ⋅ armijo_condition )
             tₖ = σ ⋅ tₖ;
        end
        xₖ = xₖ + tₖ ⋅ dₖ;
        k  = k  + 1;
        eval_x, eval_grad = f(x, true);
        if( display )
            println("Iter: $k");
            println("Value of function: $eval_x");
            println("Curret point:");
            display(xₖ);
            println("Curret Gradient: ");
            display(eval_grad);
        end
    end
    
    return xₖ;
end

newton_method (generic function with 1 method)

In [ ]:
function secant_method( α::Float64, β::Float64, γ::Float64,
    σ::Float64, ϵ::Float64, M::Int64, H::Matrix{Float64},
    xₒ::Vector{Float64}, f::function, which::Int64 ,display::Bool
    )
    # which = {0, 1}
    # if which is 0: CP1 selected
    N = length(x₀);
    k = 0;
    eval_function, eval_grad = f(xₒ, true);
    xₖ = x₀;
    Hₖ = H;
    while( norm( eval_grad, Inf ) > ϵ && k < M )
        descent_direction = linalg.solve(Hₖ, eval_grad);
        descent_direction = -1 ⋅ descent_direction;
        norm_desc = norm( dₖ, 2 );
        norm_grad = norm( eval_grad, 2 );
        if( dot(eval_grad, dₖ) > -γ⋅norm_desc⋅norm_grad )
            dₖ = -eval_grad;
            norm_desc = norm_grad;
            Hₖ = I(x₀)
        end
        if( norm_desc < β ⋅ norm_grad )
            dₖ = β ⋅ ( norm_grad / norm_desc ) ⋅ dₖ;
        end
        
        
    end
end